In [1]:
import os

In [2]:

%pwd

'c:\\Users\\asus\\OneDrive\\Desktop\\Deep-Learning-Project-Using-MLOPS-DVC\\research'

In [3]:

os.chdir("../")

In [ ]:
from dataclasses import dataclass
from pathlib import Path



@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate: float 


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [ ]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    




    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE
        )

        return training_config

In [7]:
import time

In [8]:
import os
import time
import tensorflow as tf
import numpy as np
from pathlib import Path

# ✅ Ensure eager execution
tf.config.run_functions_eagerly(True)


class PrepareCallback:
    def __init__(self, config):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [ ]:
class Training:
    def __init__(self, config):
        self.config = config
        self.model = None  # Ensure model is initialized

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

        # ✅ Fix: Reinitialize the optimizer with the correct learning rate
        optimizer = tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate)

        # ✅ Fix: Recompile the model with the new optimizer
        self.model.compile(
            optimizer=optimizer,
            loss="categorical_crossentropy",  # Change if needed
            metrics=["accuracy"]
        )

    def train_valid_generator(self):
        """Creates training and validation data generators."""
        datagenerator_kwargs = dict(
            rescale=1. / 255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

        # ✅ Debugging: Check if generators work properly
        batch = next(iter(self.train_generator))
        print("Batch type:", type(batch))  # Should be tuple (images, labels)
        print("Image batch shape:", batch[0].shape)  # Check image shape
        print("Label batch shape:", batch[1].shape)  # Check label shape

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        """Saves the trained model."""
        model.save(path)

    def train(self, callback_list: list):
        """Trains the model using generators."""
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [11]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2025-04-03 15:11:22,948: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-03 15:11:22,956: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-03 15:11:22,959: INFO: common: created directory at: artifacts]
[2025-04-03 15:11:22,964: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-04-03 15:11:22,968: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-04-03 15:11:22,972: INFO: common: created directory at: artifacts\training]


[2025-04-03 15:11:23,553: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


AttributeError: 'TrainingConfig' object has no attribute 'params_learning_rate'

In [ ]:
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[11], line 16
     11     training.train(
     12         callback_list=callback_list
     13     )
     15 except Exception as e:
---> 16     raise e

Cell In[11], line 9
      7 training_config = config.get_training_config()
      8 training = Training(config=training_config)
----> 9 training.get_base_model()
     10 training.train_valid_generator()
     11 training.train(
     12     callback_list=callback_list
     13 )

Cell In[10], line 11
      8 self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
     10 # ✅ Fix: Reinitialize the optimizer
---> 11 optimizer = tf.keras.optimizers.Adam(learning_rate=self.config.params_learning_rate)
     13 # ✅ Fix: Recompile the model to match new optimizer and training settings
     14 self.model.compile(
     15 optimizer=optimizer,
     16 loss="categorical_crossentropy",  # Change as per your use case
     17 metrics=["accuracy"]
     18 )

AttributeError: 'TrainingConfig' object has no attribute 'params_learning_rate'